In [1]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-mfkvd3w_
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-mfkvd3w_
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0df2d73d0013db7de629602c23cc120219b4f2b8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6172791 sha256=e5f24b6dfc421c743b82e9ad1324c

In [1]:
import detectron2
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

print("Detectron2 is successfully installed and imports are working.")

Detectron2 is successfully installed and imports are working.


In [2]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12

In [1]:
# !pip install ultralytics==8.0.196
# !pip install roboflow
api_key = os.getenv("ROBOFLOW_API_KEY")

from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace("food-detection-picix").project("food-detection-final-j75rs")
version = project.version(1)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Food-Detection-Final-1 in coco:: 100%|██████████| 6254/6254 [00:01<00:00, 4717.58it/s]


In [2]:
# The dataset.location should contain the COCO formatted dataset now
train_json = f"{dataset.location}/train/_annotations.coco.json"
val_json = f"{dataset.location}/valid/_annotations.coco.json"
test_json = f"{dataset.location}/test/_annotations.coco.json"

In [3]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

register_coco_instances("food_train", {}, train_json, f"{dataset.location}/train")
register_coco_instances("food_val", {}, val_json, f"{dataset.location}/valid")
register_coco_instances("food_test", {}, test_json, f"{dataset.location}/test")

food_metadata = MetadataCatalog.get("food_train")
dataset_dicts = DatasetCatalog.get("food_train")

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [5]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog

# Assuming food_metadata is already defined with the dataset information
food_metadata = MetadataCatalog.get("food_train")

# Initialize the config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Dataset registration
cfg.DATASETS.TRAIN = ("food_train",)
cfg.DATASETS.TEST = ("food_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Model weights and configurations
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025

# Adjust based on your dataset size and desired number of epochs
dataset_size = 4423  # Number of images in your training dataset
batch_size = cfg.SOLVER.IMS_PER_BATCH
iterations_per_epoch = dataset_size // batch_size
number_of_epochs = 50
cfg.SOLVER.MAX_ITER = iterations_per_epoch * number_of_epochs

# Further model and training settings
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # Adjust based on your dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(food_metadata.thing_classes)  # Your number of classes




In [7]:
# Define augmentation using Detectron2's TransformGen
from detectron2.data import transforms as T

augmentation = [
    T.RandomRotation(angle=[-15, 15]),  # Rotation between -15 and +15 degrees
    T.Resize((640, 640)),               # Resize to 640x640
    T.RandomBrightness(intensity_min=0.8, intensity_max=1.2),  # Random brightness adjustment
    T.RandomContrast(intensity_min=0.8, intensity_max=1.2),    # Random contrast adjustment
    T.RandomSaturation(intensity_min=0.8, intensity_max=1.2),  # Random saturation adjustment
    T.RandomLighting(scale=0.2)                                # Random lighting adjustment
]

# Override the data loader to include the augmentations
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.engine import DefaultTrainer

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=augmentation))


In [8]:
# Train the model
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[06/02 11:31:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:01, 110MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[06/02 11:31:51 d2.engine.train_loop]: Starting training from iteration 0


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)


Streaming output truncated to the last 5000 lines.
[06/02 11:51:57 d2.utils.events]:  eta: 2:58:04  iter: 11019  total_loss: 0.3264  loss_cls: 0.155  loss_box_reg: 0.125  loss_rpn_cls: 0.001396  loss_rpn_loc: 0.00362    time: 0.1080  last_time: 0.1123  data_time: 0.0040  last_data_time: 0.0042   lr: 0.00025  max_mem: 1150M
[06/02 11:51:59 d2.utils.events]:  eta: 2:58:01  iter: 11039  total_loss: 0.2532  loss_cls: 0.1635  loss_box_reg: 0.06866  loss_rpn_cls: 0.0002913  loss_rpn_loc: 0.002524    time: 0.1080  last_time: 0.1106  data_time: 0.0039  last_data_time: 0.0040   lr: 0.00025  max_mem: 1150M
[06/02 11:52:01 d2.utils.events]:  eta: 2:57:58  iter: 11059  total_loss: 0.3538  loss_cls: 0.201  loss_box_reg: 0.1373  loss_rpn_cls: 0.0002882  loss_rpn_loc: 0.002123    time: 0.1080  last_time: 0.1068  data_time: 0.0040  last_data_time: 0.0038   lr: 0.00025  max_mem: 1150M
[06/02 11:52:04 d2.utils.events]:  eta: 2:57:54  iter: 11079  total_loss: 0.2756  loss_cls: 0.1486  loss_box_reg: 0.135

In [9]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

val_evaluator = COCOEvaluator("food_val", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "food_val")
val_results = inference_on_dataset(trainer.model, val_loader, val_evaluator)
print("Validation Results:", val_results)

WARNING [06/02 14:52:36 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [06/02 14:52:36 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/02 14:52:36 d2.data.datasets.coco]: Loaded 1216 images in COCO format from /content/Food-Detection-Final-1/valid/_annotations.coco.json
[06/02 14:52:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/02 14:52:36 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[06/02 14:52:36 d2.data.common]: Serializing 1216 elements to byte tensors and concatenating them all ...
[06/02 14:52:36 d2.data.common]: Serialized dataset takes 0.36 MiB
[06/02 14:52:36 d2.evaluation.evaluator]: Start inference on 1216 batches


os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


[06/02 14:52:36 d2.evaluation.evaluator]: Inference done 11/1216. Dataloading: 0.0011 s/iter. Inference: 0.0343 s/iter. Eval: 0.0003 s/iter. Total: 0.0357 s/iter. ETA=0:00:43
[06/02 14:52:41 d2.evaluation.evaluator]: Inference done 149/1216. Dataloading: 0.0013 s/iter. Inference: 0.0347 s/iter. Eval: 0.0003 s/iter. Total: 0.0364 s/iter. ETA=0:00:38
[06/02 14:52:47 d2.evaluation.evaluator]: Inference done 286/1216. Dataloading: 0.0014 s/iter. Inference: 0.0349 s/iter. Eval: 0.0003 s/iter. Total: 0.0366 s/iter. ETA=0:00:33
[06/02 14:52:52 d2.evaluation.evaluator]: Inference done 422/1216. Dataloading: 0.0014 s/iter. Inference: 0.0350 s/iter. Eval: 0.0003 s/iter. Total: 0.0367 s/iter. ETA=0:00:29
[06/02 14:52:57 d2.evaluation.evaluator]: Inference done 558/1216. Dataloading: 0.0014 s/iter. Inference: 0.0350 s/iter. Eval: 0.0003 s/iter. Total: 0.0367 s/iter. ETA=0:00:24
[06/02 14:53:02 d2.evaluation.evaluator]: Inference done 695/1216. Dataloading: 0.0014 s/iter. Inference: 0.0350 s/iter. 

In [10]:
# Evaluate on the test dataset
cfg.DATASETS.TEST = ("food_test",)
test_evaluator = COCOEvaluator("food_test", cfg, False, output_dir="./output")
test_loader = build_detection_test_loader(cfg, "food_test")
test_results = inference_on_dataset(trainer.model, test_loader, test_evaluator)
print("Test Results:", test_results)

WARNING [06/02 15:07:56 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [06/02 15:07:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/02 15:07:56 d2.data.datasets.coco]: Loaded 608 images in COCO format from /content/Food-Detection-Final-1/test/_annotations.coco.json
[06/02 15:07:56 d2.data.build]: Distribution of instances among all 18 categories:
|  category   | #instances   |   category    | #instances   |   category    | #instances   |
|:-----------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| foods-Foods | 0            | Braised dish  | 31           | Chinese pan.. | 70           |
| Fried food  | 65           | Northern cake | 46           |    Pickles    | 66           |
| Steamed bun | 262          | Stir-fried .. | 44           |      bao      | 23           |
|  box

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


[06/02 15:07:57 d2.evaluation.evaluator]: Inference done 11/608. Dataloading: 0.0013 s/iter. Inference: 0.0354 s/iter. Eval: 0.0003 s/iter. Total: 0.0371 s/iter. ETA=0:00:22
[06/02 15:08:02 d2.evaluation.evaluator]: Inference done 149/608. Dataloading: 0.0014 s/iter. Inference: 0.0345 s/iter. Eval: 0.0003 s/iter. Total: 0.0363 s/iter. ETA=0:00:16
[06/02 15:08:07 d2.evaluation.evaluator]: Inference done 287/608. Dataloading: 0.0014 s/iter. Inference: 0.0346 s/iter. Eval: 0.0003 s/iter. Total: 0.0364 s/iter. ETA=0:00:11
[06/02 15:08:12 d2.evaluation.evaluator]: Inference done 427/608. Dataloading: 0.0014 s/iter. Inference: 0.0344 s/iter. Eval: 0.0003 s/iter. Total: 0.0362 s/iter. ETA=0:00:06
[06/02 15:08:17 d2.evaluation.evaluator]: Inference done 568/608. Dataloading: 0.0014 s/iter. Inference: 0.0343 s/iter. Eval: 0.0003 s/iter. Total: 0.0361 s/iter. ETA=0:00:01
[06/02 15:08:19 d2.evaluation.evaluator]: Total inference time: 0:00:21.822568 (0.036190 s / iter per device, on 1 devices)
[0

In [11]:
# Output model parameters
model_parameters = trainer.model.state_dict()
for param_tensor in model_parameters:
    print(f"{param_tensor}: {model_parameters[param_tensor].size()}")

backbone.fpn_lateral2.weight: torch.Size([256, 256, 1, 1])
backbone.fpn_lateral2.bias: torch.Size([256])
backbone.fpn_output2.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output2.bias: torch.Size([256])
backbone.fpn_lateral3.weight: torch.Size([256, 512, 1, 1])
backbone.fpn_lateral3.bias: torch.Size([256])
backbone.fpn_output3.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output3.bias: torch.Size([256])
backbone.fpn_lateral4.weight: torch.Size([256, 1024, 1, 1])
backbone.fpn_lateral4.bias: torch.Size([256])
backbone.fpn_output4.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output4.bias: torch.Size([256])
backbone.fpn_lateral5.weight: torch.Size([256, 2048, 1, 1])
backbone.fpn_lateral5.bias: torch.Size([256])
backbone.fpn_output5.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output5.bias: torch.Size([256])
backbone.bottom_up.stem.conv1.weight: torch.Size([64, 3, 7, 7])
backbone.bottom_up.stem.conv1.norm.weight: torch.Size([64])
backbone.bottom_up.stem.conv1.norm.bias: t